In [18]:
import os
import numpy as np
import emu.neuralynx_io as nlx
from pynwb.misc import AnnotationSeries
from pynwb import NWBFile
import pandas as pd
import datetime

In [2]:
proj_root = '/home/elijahc/.emu/'

seeg_root = os.path.join(proj_root,'pdil','pt_01','SEEG')
print(seeg_root)

/home/elijahc/.emu/pdil/pt_01/SEEG


In [3]:
import glob

In [15]:
def load_all_nev(path):
    nev_files = sorted(glob.glob(os.path.join(path,'*.nev')))
    nev1 = nlx.load_nev(nev_files[0])['events']
    cols = nev1.dtype.names
    for p in nev_files:
        f = nlx.load_nev(p)['events']
        for row in f:
            r = {k:v for k,v in zip(cols,row)}
            r['path'] = os.path.split(p)[-1]
            yield r
         
def df_from_raw(path,index=None):
    ev = pd.DataFrame.from_records(load_all_nev(path),index=index)
    ev['EventString'] = [str(v,'utf-8') for v in ev.EventString.values]
    ev['time'] = pd.to_datetime(ev.TimeStamp.values,unit='us')
    ev = ev.set_index('TimeStamp')
    return ev

def label_blockstart(df,threshold=220000):
    labels = ['block_start']
    for i in np.arange(len(df.index.values))[1:-1]:
#         triplet = df.index.values[i-1:i+1]
        diff_prev = int(np.abs(df.index.values[i]-df.index.values[i-1]))
        diff_next = int(np.abs(df.index.values[i]-df.index.values[i+1]))
#         print([diff_prev, diff_next])
        if  diff_prev <= threshold or diff_next <= threshold:
            labels.append('block_start')
            j=1
#         elif i<len(df.index.values) and  <= threshold:
#             labels.append('block_start')
        else:
            labels.append('trial_start'.format(int(j)))
            j+=1
    labels.append('trial_start'.format(str(j)))
    df['label'] = labels
    return df

In [ ]:
pd.to_datetime()

In [56]:
nlx.load_nev(os.path.join(seeg_root,'raw','Events_0007.nev'))

/home/elijahc/dev/emu/emu/neuralynx_io.py:79: UserWarning: Unable to parse original file path from Neuralynx header: -FileType Event
  warnings.warn('Unable to parse original file path from Neuralynx header: ' + hdr_lines[1])
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileVersion 3.2
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileUUID d11594df-8b47-4683-b9fa-6f4f452b31ce
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line from Neuralynx header: -TimeCreated 2020/03/04 13:35:45
  warnings.warn('Unable to parse parameter line from Neuralynx header: ' + line)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line f

{'file_path': '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/Events_0007.nev',
 'raw_header': b'######## Neuralynx Data File Header\r\n-FileType Event\r\n-FileVersion 3.2\r\n-FileUUID d11594df-8b47-4683-b9fa-6f4f452b31ce\r\n-SessionUUID 6f2002cc-7de0-4456-997d-610326db2453\r\n-OriginalFileName D:\\PegasusData\\2020-03-03_09-18-11\\Events_0007.nev\r\n-TimeCreated 2020/03/04 13:35:45\r\n-TimeClosed 2020/03/06 12:30:46\r\n-RecordSize 184\r\n-ApplicationName Pegasus "2.1.2 "\r\n-AcquisitionSystem AcqSystem1 ATLAS\r\n-AcqEntName Events\r\n',
 'header': {'TimeOpened': 'leVersion 3.2',
  'TimeOpened_dt': None,
  'TimeClosed': 'leUUID d11594df-8b47-4683-b9fa-6f4f452b31ce',
  'TimeClosed_dt': None,
  'SessionUUID': '6f2002cc-7de0-4456-997d-610326db2453',
  'OriginalFileName': 'D:\\PegasusData\\2020-03-03_09-18-11\\Events_0007.nev',
  'RecordSize': '184',
  'AcqEntName': 'Events'},
 'records': array([(0, 0, 2, 1583328945020845, 19, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'Starting Recording'),
        (

In [19]:
events = load_all_nev(os.path.join(seeg_root,'raw'))
df = df_from_raw(os.path.join(seeg_root,'raw'))

# Extract only events labeled ttl
# df = df[df.EventString=='ttl']

# Add ttl labels
# df = label_blockstart(df)

/home/elijahc/dev/emu/emu/neuralynx_io.py:79: UserWarning: Unable to parse original file path from Neuralynx header: -FileType Event
  warnings.warn('Unable to parse original file path from Neuralynx header: ' + hdr_lines[1])
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileVersion 3.2
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileUUID d11594df-8b47-4683-b9fa-6f4f452b31ce
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line from Neuralynx header: -TimeCreated 2020/03/04 13:35:45
  warnings.warn('Unable to parse parameter line from Neuralynx header: ' + line)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line f

In [59]:
ts = pd.to_datetime(df.index.values,unit='us')[0]

In [70]:
df.time.iloc[0].to_pydatetime()

datetime.datetime(2020, 3, 4, 13, 35, 45, 20845)

In [53]:
start_time = datetime.datetime.fromtimestamp(df.index.values.min()/1000/1000)

In [28]:
# start_time = datetime.datetime(2020,3,4,13,38,19)

In [55]:
NWBFile(session_description='Post-Op Day 2', identifier='0007',session_start_time=start_time)

/usr/local/lib/python3.6/dist-packages/pynwb/file.py:691: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


root pynwb.file.NWBFile at 0x140346414605480
Fields:
  file_create_date: [datetime.datetime(2020, 3, 11, 20, 22, 31, 172355, tzinfo=tzlocal())]
  identifier: 0007
  session_description: Post-Op Day 2
  session_start_time: 2020-03-04 06:35:45.020845-07:00
  timestamps_reference_time: 2020-03-04 06:35:45.020845-07:00

In [11]:
block_ttls = df[df.label=='block_start']


In [13]:
annos = AnnotationSeries(name='block_start_ttl', data=block_ttls.label.values, timestamps=block_ttls.index.values/1000/1000)

In [14]:
annos

block_start_ttl pynwb.misc.AnnotationSeries at 0x140346422209280
Fields:
  comments: no comments
  conversion: 1.0
  data: ['block_start' 'block_start' 'block_start' 'block_start' 'block_start'
 'block_start']
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [1.58332911e+09 1.58332911e+09 1.58332944e+09 1.58332944e+09
 1.58332990e+09 1.58332990e+09]
  timestamps_unit: seconds
  unit: n/a

In [ ]:
lfp = nlx.load_ncs(ncs_path)

In [ ]:
lfp['time'].shape

In [ ]:
lfp['data'].shape

In [ ]:
lfp_df = pd.DataFrame.from_records({'time':lfp['time'],'data':lfp['data']},index='time')

In [ ]:
lfp.keys()

In [ ]:
lfp_df.iloc[:13*4000].plot()

In [ ]:
lfp.keys()

In [ ]:
paths = load_all_nev(os.path.join(seeg_root,'raw'))
paths

In [ ]:
np.nargmax([len(nlx.load_nev(p)['records']) for p in paths])

In [ ]:
# df = pd.DataFrame.from_records(recs)
df

# events_df = pd.DataFrame.from_records(list(nev['events'][0]),columns=nev['events'].dtype.names)

In [ ]:
recs